In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders import LeaveOneOutEncoder
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv('info.Allergen_Status_of_Food_Products.csv', keep_default_na=False, na_values="")

# Ensure the dataset has the required columns
print("Columns in dataset:", data.columns)

# Check the percentage of missing values for each column
missing_percentage = data.isnull().mean() * 100
print("Percentage of Missing Values:\n", missing_percentage)

# Strategy: Drop columns with >50% missing values, fill others
threshold = 50  # Define a threshold for acceptable missing values
columns_to_drop = missing_percentage[missing_percentage > threshold].index
print("\nColumns to drop (more than 50% missing):", columns_to_drop)

# Drop columns with too many missing values
data.drop(columns=columns_to_drop, inplace=True)

# Fill missing values for numerical columns with mean
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[numerical_cols] = data[numerical_cols].fillna(data[numerical_cols].mean())

# Fill missing values for categorical columns with mode
categorical_cols = data.select_dtypes(include=['object']).columns
data[categorical_cols] = data[categorical_cols].fillna(data[categorical_cols].mode().iloc[0])

# Verify no missing values remain
print("\nMissing values after handling:")
print(data.isnull().sum())


# Split into train and test datasets
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Map the Prediction column to numerical values
if 'Prediction' in train.columns:
    train['Prediction'] = train['Prediction'].map({'Contains': 1, 'Does not contain': 0})
    test['Prediction'] = test['Prediction'].map({'Contains': 1, 'Does not contain': 0})
else:
    print("Error: 'Prediction' column is missing.")

# Identify categorical columns in the training data
categorical_columns_train = train.select_dtypes(include=['object']).columns

# Initialize the Leave-One-Out Encoder
encoder = LeaveOneOutEncoder(cols=categorical_columns_train)

# Fit the encoder on the training data and transform it
train_encoded = encoder.fit_transform(train[categorical_columns_train], train["Prediction"])

# Transform the test data using the fitted encoder
test_encoded = encoder.transform(test[categorical_columns_train])

# Add the encoded columns back to the original datasets
train = pd.concat([train.drop(categorical_columns_train, axis=1), train_encoded], axis=1)
test = pd.concat([test.drop(categorical_columns_train, axis=1), test_encoded], axis=1)

# Check the resulting datasets
print("Training set shape:", train.shape)
print("Test set shape:", test.shape)


Columns in dataset: Index(['Food Product', 'Main Ingredient', 'Sweetener', 'Fat/Oil', 'Seasoning',
       'Allergens', 'Price ($)', 'Customer rating (Out of 5)', 'Prediction'],
      dtype='object')
Percentage of Missing Values:
 Food Product                  0.000000
Main Ingredient               0.000000
Sweetener                     0.000000
Fat/Oil                       0.000000
Seasoning                     0.000000
Allergens                     0.000000
Price ($)                     0.000000
Customer rating (Out of 5)    0.000000
Prediction                    0.250627
dtype: float64

Columns to drop (more than 50% missing): Index([], dtype='object')

Missing values after handling:
Food Product                  0
Main Ingredient               0
Sweetener                     0
Fat/Oil                       0
Seasoning                     0
Allergens                     0
Price ($)                     0
Customer rating (Out of 5)    0
Prediction                    0
dtype: int64
Tra

In [20]:
test

,Price ($),Customer rating (Out of 5),Prediction,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens
198,6.26,3.0,0,0.62069,0.533333,0.521930,0.383562,0.62069,0.03200
349,11.87,1.3,1,1.00000,0.666667,0.876712,0.927536,1.00000,1.00000
33,8.67,4.9,1,0.62069,1.000000,0.521930,0.927536,0.62069,1.00000
208,5.65,3.9,0,0.62069,0.000000,0.521930,0.383562,0.25000,0.03200
93,13.79,2.2,1,0.62069,0.620690,0.521930,0.630435,0.62069,0.62069
...,...,...,...,...,...,...,...,...,...
249,7.87,4.3,0,0.62069,0.400000,0.521930,0.383562,0.62069,0.03200
225,6.28,3.1,0,0.62069,0.533333,0.521930,0.383562,1.00000,0.03200
368,5.16,1.3,1,1.00000,0.545455,0.521930,0.630435,1.00000,1.00000
175,10.32,3.8,0,0.62069,0.666667,0.620690,0.630435,0.37500,0.03200


In [21]:
# Split features and target variable for training and test sets
X_train = train.drop(columns=['Prediction'])  # Features
y_train = train['Prediction']                # Target

X_test = test.drop(columns=['Prediction'])   # Features
y_test = test['Prediction']                  # Target

# Verify the shapes of features and targets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (319, 8)
y_train shape: (319,)
X_test shape: (80, 8)
y_test shape: (80,)


In [22]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [23]:
# Initialize the XGBoost classifier
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Train the model on the training data
xgb_model.fit(X_train, y_train)


C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:23:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [24]:
# Make predictions on training and test sets
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)


In [25]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Calculate accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Classification reports
print("\nClassification Report - Train:\n", classification_report(y_train, y_train_pred))
print("\nClassification Report - Test:\n", classification_report(y_test, y_test_pred))

# Confusion matrices
print("\nConfusion Matrix - Train:\n", confusion_matrix(y_train, y_train_pred))
print("\nConfusion Matrix - Test:\n", confusion_matrix(y_test, y_test_pred))


Train Accuracy: 1.0000
Test Accuracy: 0.9750

Classification Report - Train:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       121
           1       1.00      1.00      1.00       198

    accuracy                           1.00       319
   macro avg       1.00      1.00      1.00       319
weighted avg       1.00      1.00      1.00       319


Classification Report - Test:
               precision    recall  f1-score   support

           0       0.95      0.95      0.95        22
           1       0.98      0.98      0.98        58

    accuracy                           0.97        80
   macro avg       0.97      0.97      0.97        80
weighted avg       0.97      0.97      0.97        80


Confusion Matrix - Train:
 [[121   0]
 [  0 198]]

Confusion Matrix - Test:
 [[21  1]
 [ 1 57]]


In [26]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Cross-validation Accuracy: {cv_scores.mean():.4f}")


C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:23:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:23:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:23:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\S

Cross-validation Accuracy: 0.9874


In [27]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
}

# Initialize GridSearchCV
grid_search = GridSearchCV(XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
                           param_grid, cv=5, scoring='accuracy', verbose=1)

# Fit GridSearchCV on training data
grid_search.fit(X_train, y_train)

# Print the best parameters and accuracy
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-validation Accuracy:", grid_search.best_score_)


C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:23:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:23:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:23:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\S

Fitting 5 folds for each of 54 candidates, totalling 270 fits


C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:23:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:23:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:23:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\S

Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Best Cross-validation Accuracy: 0.9874007936507937


In [28]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
}

# Perform Grid Search CV
grid_search = GridSearchCV(
    estimator=XGBClassifier(eval_metric='logloss', random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1
)

grid_search.fit(X_train, y_train_encoded)

# Print the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-validation Accuracy:", grid_search.best_score_)


Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Best Cross-validation Accuracy: 0.9874007936507937


In [29]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Make predictions on the training set
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)

# Calculate accuracy for training and test sets
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Classification reports for detailed performance metrics
print("\nClassification Report - Train:\n", classification_report(y_train, y_train_pred))
print("\nClassification Report - Test:\n", classification_report(y_test, y_test_pred))

# Confusion matrices for training and test sets
print("\nConfusion Matrix - Train:\n", confusion_matrix(y_train, y_train_pred))
print("\nConfusion Matrix - Test:\n", confusion_matrix(y_test, y_test_pred))


Train Accuracy: 1.0000
Test Accuracy: 0.9750

Classification Report - Train:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       121
           1       1.00      1.00      1.00       198

    accuracy                           1.00       319
   macro avg       1.00      1.00      1.00       319
weighted avg       1.00      1.00      1.00       319


Classification Report - Test:
               precision    recall  f1-score   support

           0       0.95      0.95      0.95        22
           1       0.98      0.98      0.98        58

    accuracy                           0.97        80
   macro avg       0.97      0.97      0.97        80
weighted avg       0.97      0.97      0.97        80


Confusion Matrix - Train:
 [[121   0]
 [  0 198]]

Confusion Matrix - Test:
 [[21  1]
 [ 1 57]]


In [30]:
from sklearn.model_selection import cross_val_score
import numpy as np

# Perform cross-validation
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='accuracy')

# Print the mean and standard deviation of cross-validation scores
print(f"Cross-validation accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")


C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:29:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:29:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Shubham Sonawane\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:29:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\S

Cross-validation accuracy: 0.9874 ± 0.0119


In [32]:
import joblib

# Save the trained XGBoost model
joblib.dump(xgb_model, 'ngsxgb_model.pkl')

# Save the Leave-One-Out Encoder
joblib.dump(encoder, 'ngsloo_encoder.pkl')
print("model abd encoder saved succesfully")

model abd encoder saved succesfully


In [33]:
# Get feature importance as a dataframe
importance = xgb_model.get_booster().get_score(importance_type='weight')

# Convert to a dataframe for easier viewing
import pandas as pd
importance_df = pd.DataFrame(importance.items(), columns=['Feature', 'Importance'])
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display the top 10 features
print(importance_df.head(10))


                      Feature  Importance
0                   Price ($)        46.0
2             Main Ingredient        32.0
4                     Fat/Oil        28.0
3                   Sweetener        25.0
1  Customer rating (Out of 5)        24.0
6                   Allergens        16.0
5                   Seasoning        10.0
